In [1]:
using Revise

In [2]:
using MeshCatMechanisms
using MeshCat
using RigidBodyDynamics
using RigidBodyDynamics.Contact
using LCPSim
using Blink
using LearningMPC
using LearningMPC.Models
using RigidBodySim
using CoordinateTransformations
using IKMimic
using MathOptInterface
const MOI = MathOptInterface

HTML{String}("    <script class='js-collapse-script'>\n        var curMatch =\n            window.location.href\n            .match(/(.*?)\\/notebooks\\/.*\\.ipynb/);\n\n        curMatch = curMatch ||\n            window.location.href\n            .match(/(.*?)\\/apps\\/.*\\.ipynb/);\n\n        if ( curMatch ) {\n            \$('head').append('<base href=\"' + curMatch[1] + '/\">');\n        }\n    </script>\n")

HTML{String}("<script class='js-collapse-script' src='/assetserver/9db0e137604e898c25f194d972715156bd46cb81-assets/webio/dist/bundle.js'></script>")

HTML{String}("<script class='js-collapse-script' src='/assetserver/9db0e137604e898c25f194d972715156bd46cb81-assets/providers/ijulia_setup.js'></script>")

HTML{String}("  <script class='js-collapse-script'>\n    \$('.js-collapse-script').parent('.output_subarea').css('padding', '0');\n  </script>\n")

INFO: Loading HttpServer methods...
INFO: Recompiling stale cache file /home/rdeits/locomotion/explorations/learning-mpc-2/packages/lib/v0.6/OSQP.ji for module OSQP.
INFO: Recompiling stale cache file /home/rdeits/locomotion/explorations/learning-mpc-2/packages/lib/v0.6/LearningMPC.ji for module LearningMPC.


MathOptInterface

In [3]:
boxatlas = BoxAtlas()
atlas = LearningMPC.Models.PlanarAtlas(:simulation);

In [4]:
vis = Visualizer()
open(vis, Window())
mvis_b = MechanismVisualizer(boxatlas, vis[:boxatlas])
mvis_a = MechanismVisualizer(atlas, vis[:atlas])
settransform!(vis[:boxatlas], Translation(-1, 0, 0))
set_configuration!(mvis_a, configuration(nominal_state(atlas)))
set_configuration!(mvis_b, configuration(nominal_state(boxatlas)))

AtlasRobot.packagepath() = "/home/rdeits/locomotion/explorations/learning-mpc-2/packages/v0.6/AtlasRobot/deps"


In [5]:
lqr_b = LQRSolution(boxatlas)

using Base.Iterators: product

# xstar = nominal_state(atlas)
# Δt = 0.05
# nq = num_positions(xstar)
# nv = num_velocities(xstar)
# qq = fill(0.1, nq)
# qq[1] = 10
# qq[2] = 100
# qq[3] = 500
# # qq[configuration_range(xstar, findjoint(mechanism(atlas_sim), "r_leg_hpx"))] .= 100
# # qq[configuration_range(xstar, findjoint(mechanism(atlas_sim), "l_leg_hpx"))] .= 100
# qv = fill(0.01, nv)
# qv[velocity_range(xstar, findjoint(xstar.mechanism, "floating_base"))] .= [100, 20, 50]
# Q = diagm(vcat(qq, qv))
# R = diagm(fill(1e-6, nv))
# lqr_a = LearningMPC.LQRSolution(xstar, Q, R, Δt, 
#     [Point3D(default_frame(body), 0., 0, 0) for body in 
#             findbody.(mechanism(atlas), ["r_foot_sole", "l_foot_sole"])])

Academic license - for non-commercial use only
Academic license - for non-commercial use only


In [96]:
function build_contact_controller(mechanism, lqrsol, contacts, contact_state)
    lowlevel = QPControl.MPCController{4}(mechanism, LearningMPC.defaultoptimizer())
    stage = QPControl.addstage!(lowlevel, 0.01)
    for (i, (point, surface)) in enumerate(contacts)
        body = body_fixed_frame_to_body(mechanism, point.frame)
        contact_point = QPControl.addcontact!(stage,
            body,
            point,
            FreeVector3D(default_frame(body), 0., 0, 1),
            0.8,
            surface
        )
        if contact_state[i]
            contact_point.maxnormalforce = 1e9
        else
            contact_point.maxnormalforce = 0
        end
    end
    lowlevel
end

function build_mixed_controller(mechanism, lqrsol, contacts::AbstractVector{<:Tuple{Point3D, HalfSpace3D}})

    matching_bodies =  ["pelvis", "r_foot_sole", "l_foot_sole", "r_hand_mount", "l_hand_mount"]
    ik_state_b = MechanismState(boxatlas.mechanism)
    ik_state_a = MechanismState(atlas.mechanism)
    ik_work = IKMimic.IKMimicWorkspace(ik_state_a, ik_state_b, matching_bodies)
    ik_sensitivity_work = IKMimic.SensitivityWorkspace{Float64}(
        num_positions(ik_state_a) + num_velocities(ik_state_a),
        num_positions(ik_state_b) + num_velocities(ik_state_b)
    )
    
    contact_states = collect(product([[true, false] for _ in contacts]...))
    @show vec(contact_states)
#     contact_states = [[true, true]]
    controllers = map(contact_states) do contact_state
        build_contact_controller(mechanism, lqrsol, contacts, contact_state)
    end
    τs = [zeros(num_velocities(mechanism)) for controller in controllers]
    objective_values = zeros(length(controllers))

    function control(τ, t, x)
        copy!(ik_state_a, x)
        for i in 1:10
            IKMimic.ik_mimic!(ik_state_b, ik_state_a, ik_work)
            set_configuration!(ik_state_b, 
                clamp.(configuration(ik_state_b), LCPSim.all_configuration_bounds(ik_state_b.mechanism))
            )
        end
        IKMimic.sensitivity!(ik_sensitivity_work, ik_work)
        copy!(mvis_b, ik_state_b)
        copy!(mvis_a, x)
        
        let controllers = controllers,
            τs = τs,
            objective_values = objective_values

#             Threads.@threads
            for i in eachindex(controllers)
                controller = controllers[i]
                # controller.running_state_cost.x0[2] = 0.7 + 0.2 * sin(t)
                # controller.terminal_state_cost.x0[2] = 0.7 + 0.2 * sin(t)
                if !controller.initialized
                    QPControl.initialize!(controller)
                end
                controller.running_input_cost.Q .= Diagonal(fill(1e-5, num_velocities(x)))
                controller.running_input_cost.q .= 0
                controller.running_input_cost.x0 .= 0
                
                J_ba = ik_sensitivity_work.J
#                 @show J_ba
                x_a0 = vcat(configuration(x), velocity(x))
                x_b0 = vcat(configuration(ik_state_b), velocity(ik_state_b))
                x_bstar = lqrsol.x0
#                 @show x_a0
#                 @show x_b0
#                 @show x_bstar
                
                controller.running_state_cost.Q .= J_ba' * lqrsol.Q * J_ba
                controller.running_state_cost.q .= (
                    (2 * (-J_ba * x_a0 + x_b0 - x_bstar)' * lqrsol.Q' * J_ba)')
                controller.running_state_cost.x0 .= 0
                controller.running_state_cost.constant[] = (
                    (-J_ba * x_a0 + x_b0 - x_bstar)' * lqrsol.Q * (-J_ba * x_a0 + x_b0 - x_bstar))
                
                controller.terminal_state_cost.Q .= J_ba' * lqrsol.S * J_ba
                controller.terminal_state_cost.q .= (
                    (2 * (-J_ba * x_a0 + x_b0 - x_bstar)' * lqrsol.S' * J_ba)')
                controller.terminal_state_cost.x0 .= 0
                controller.terminal_state_cost.constant[] = (
                    (-J_ba * x_a0 + x_b0 - x_bstar)' * lqrsol.S * (-J_ba * x_a0 + x_b0 - x_bstar))
                
                for stage in QPControl.stages(controller)
                    QPControl.copyto!(stage.state, x)
                end
                model = controller.qpmodel
                SimpleQP.solve!(model)
#                 for stage in QPControl.stages(controller)
#                     for (body, contacts) in stage.contacts
#                         for (point, surface) in contacts
#                             @show SimpleQP.value.(model, point.wrench_world)
#                         end
#                     end
#                 end
                τs[i] .= SimpleQP.value.(model, controller.stages[1].input)

                @show SimpleQP.primalstatus(model)
                if SimpleQP.primalstatus(model) ∈ (MOI.UnknownResultStatus, MOI.InfeasiblePoint, MOI.InfeasibilityCertificate, MOI.InfeasibleOrUnbounded)
                    objective_values[i] = Inf
                else
                    objective_values[i] = SimpleQP.objectivevalue(model)
                end
            end
        end
        best_cost = Inf
        best_model = 0
        for i in eachindex(controllers)
            if objective_values[i] < best_cost
                best_model = i
                best_cost = objective_values[i]
                τ .= τs[i]
            end
        end
        @show collect(zip(vec(contact_states), objective_values))
#         @show objective_values
#         @show best_cost
        if best_cost == Inf
            @show configuration(x) velocity(x)
            set_configuration!(mvis_a, configuration(x))
            error("infeasible")
        end
        @show best_model contact_states[best_model]
        τ
    end, controllers
end

mixed_control, lowlevel_controllers = build_mixed_controller(mechanism(atlas), lqr_b,
    vcat(
        [
            (Point3D(default_frame(findbody(mechanism(atlas), bodyname)), 0., 0, 0),
             HalfSpace3D(Point3D(root_frame(mechanism(atlas)), 0., 0., 0),
                         FreeVector3D(root_frame(mechanism(atlas)), 0., 0, 1)))
            for bodyname in ["r_foot_sole", "l_foot_sole"]
        ],
#         [
#             (Point3D(default_frame(findbody(mechanism(boxatlas), "l_hand_mount")), 0., 0, 0),
#              HalfSpace3D(Point3D(root_frame(mechanism(boxatlas)), 0., 1., 0),
#                          FreeVector3D(root_frame(mechanism(boxatlas)), 0., -1, 0)))
#         ]
    ));

qp = lowlevel_controllers[1].qpmodel

vec(contact_states) = Tuple{Bool,Bool}[(true, true), (false, true), (true, false), (false, false)]
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only


Model{Float64, Gurobi.GurobiOptimizer}(…)

In [97]:
state = nominal_state(atlas)
# set_velocity!(state, findjoint(mechanism(atlas_sim), "floating_base"), [-0., 0, 0])
mixed_control(zeros(num_velocities(state)), 0.0, state)

SimpleQP.primalstatus(model) = FeasiblePoint::MathOptInterface.ResultStatusCode = 0
SimpleQP.primalstatus(model) = FeasiblePoint::MathOptInterface.ResultStatusCode = 0
SimpleQP.primalstatus(model) = FeasiblePoint::MathOptInterface.ResultStatusCode = 0
SimpleQP.primalstatus(model) = FeasiblePoint::MathOptInterface.ResultStatusCode = 0
collect(zip(vec(contact_states), objective_values)) = Tuple{Tuple{Bool,Bool},Float64}[((true, true), 7.07952), ((false, true), 7.04154), ((true, false), 7.04172), ((false, false), 7.16971)]
best_model = 2
contact_states[best_model] = (false, true)


23-element Array{Float64,1}:
    0.0       
    0.0       
    0.0       
   85.4334    
   -1.06508   
  100.943     
   63.6383    
    5.7558    
   -8.28769   
 -206.737     
   46.5737    
  -15.5038    
   11.6547    
   97.274     
    2.02922   
   -0.156485  
   -0.237203  
   10.9277    
    0.206832  
   -7.51153   
    6.42435   
   -0.0081783 
   -0.00986548

In [98]:
state = nominal_state(atlas)
# set_velocity!(state, findjoint(mechanism(atlas), "floating_base"), [-0.3, 0, 0])
problem = LearningMPC.simulation_problem(state, mixed_control, 0.01, 0.2)
@time sol = RigidBodySim.solve(problem, Tsit5(), abs_tol = 1e-8, dt = 1e-6);
setanimation!(mvis_a, sol)

SimpleQP.primalstatus(model) = FeasiblePoint::MathOptInterface.ResultStatusCode = 0
SimpleQP.primalstatus(model) = FeasiblePoint::MathOptInterface.ResultStatusCode = 0
SimpleQP.primalstatus(model) = FeasiblePoint::MathOptInterface.ResultStatusCode = 0
SimpleQP.primalstatus(model) = FeasiblePoint::MathOptInterface.ResultStatusCode = 0
collect(zip(vec(contact_states), objective_values)) = Tuple{Tuple{Bool,Bool},Float64}[((true, true), 7.07951), ((false, true), 7.04154), ((true, false), 7.04172), ((false, false), 7.1697)]
best_model = 2
contact_states[best_model] = (false, true)
SimpleQP.primalstatus(model) = FeasiblePoint::MathOptInterface.ResultStatusCode = 0
SimpleQP.primalstatus(model) = FeasiblePoint::MathOptInterface.ResultStatusCode = 0
SimpleQP.primalstatus(model) = FeasiblePoint::MathOptInterface.ResultStatusCode = 0
SimpleQP.primalstatus(model) = FeasiblePoint::MathOptInterface.ResultStatusCode = 0
collect(zip(vec(contact_states), objective_values)) = Tuple{Tuple{Bool,Bool},Floa

LoadError: [91mGurobi.GurobiError(10020, "Objective Q not PSD (diagonal adjustment of 2.9e-06 would be required)")[39m